# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pickle

# imports for normalizing and tokenizing
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk


# imports for pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords', 'omw-1.4'])

[nltk_data] Downloading package punkt to /home/heiko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/heiko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/heiko/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/heiko/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/heiko/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_query('select * from CleanedData', engine)
X = df[['message']]
Y = df[df.columns[4:]]
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    ''' normalizes, removes stopwords and lemmatizes

    INPUT:
        text         - text in English language
    
    OUTPUT:
        clean_tokens - list of cleaned tokens
    ''' 
    
    # Remove punctuation characters
    text = re.sub('[^0-9a-z]+', ' ', text.lower())
    #Split text into words using NLTK
    words = word_tokenize(text)
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]

    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in words:
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                # ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfVectorizer(tokenizer=tokenize))
            ])),

        ])),

        ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(random_state = 42)))
    ])
# pipeline = Pipeline([
#             ('tfidf', TfidfVectorizer(tokenizer= tokenize)),
#             ('clf', MultiOutputClassifier(RandomForestClassifier(random_state = 42)))
#         ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train.message, y_train) # input for x_train must be text NOT list!




/home/heiko/miniconda3/envs/disaster/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('tfidf',
                                                                  TfidfVectorizer(tokenizer=<function tokenize at 0x7f10ce209f70>))]))])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))])

In [6]:
y_test.shape

(6554, 36)

In [7]:
X_train.shape

(19662, 1)

In [18]:
# dump model
import joblib
joblib.dump(pipeline, 'classifier.pkl')

['classifier.pkl']

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
predicted = pipeline.predict(X_test.message)

In [9]:
type(predicted)

numpy.ndarray

In [10]:
predicted.shape

(6554, 36)

In [11]:
headers=y_test.columns
headers = list(headers)
headers

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [12]:
y_test.iloc[:, 0]

21375    1
3837     1
16174    0
11688    0
11776    1
        ..
2463     1
25398    1
8929     1
4947     1
18681    0
Name: related, Length: 6554, dtype: int64

In [13]:
predicted.shape

(6554, 36)

In [14]:
predicted[0]

array([1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [15]:
for index, y_pred in enumerate(np.transpose(predicted)):
    y_true=y_test.iloc[:, index]
    print(f'{headers[index]}: {classification_report(y_true, y_pred)}')


related:               precision    recall  f1-score   support

           0       0.71      0.44      0.54      1562
           1       0.84      0.94      0.89      4949
           2       0.23      0.42      0.30        43

    accuracy                           0.82      6554
   macro avg       0.60      0.60      0.58      6554
weighted avg       0.81      0.82      0.80      6554

request:               precision    recall  f1-score   support

           0       0.90      0.98      0.94      5387
           1       0.83      0.48      0.61      1167

    accuracy                           0.89      6554
   macro avg       0.86      0.73      0.77      6554
weighted avg       0.89      0.89      0.88      6554

offer:               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6519
           1       0.00      0.00      0.00        35

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      

/home/heiko/miniconda3/envs/disaster/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/heiko/miniconda3/envs/disaster/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/heiko/miniconda3/envs/disaster/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

earthquake:               precision    recall  f1-score   support

           0       0.98      0.99      0.99      5989
           1       0.88      0.80      0.84       565

    accuracy                           0.97      6554
   macro avg       0.93      0.90      0.91      6554
weighted avg       0.97      0.97      0.97      6554

cold:               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6427
           1       0.58      0.06      0.10       127

    accuracy                           0.98      6554
   macro avg       0.78      0.53      0.55      6554
weighted avg       0.97      0.98      0.97      6554

other_weather:               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6250
           1       0.47      0.05      0.09       304

    accuracy                           0.95      6554
   macro avg       0.71      0.52      0.53      6554
weighted avg       0.93      0.95      0.

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__tfidf', 'features__text_pipeline__tfidf__analyzer', 'features__text_pipeline__tfidf__binary', 'features__text_pipeline__tfidf__decode_error', 'features__text_pipeline__tfidf__dtype', 'features__text_pipeline__tfidf__encoding', 'features__text_pipeline__tfidf__input', 'features__text_pipeline__tfidf__lowercase', 'features__text_pipeline__tfidf__max_df', 'features__text_pipeline__tfidf__max_features', 'features__text_pipeline__tfidf__min_df', 'features__text_pipeline__tfidf__ngram_range', 'features__text_pipeline__tfidf__norm', 'features__text_pipeline__tfidf__preprocessor', 'features__text_pipeline__tfidf__smooth_idf', 'features__text_pipeline__tfidf__stop_words', 'featu

# Parameters
## n_estimators
This is the number of trees you want to build before taking the maximum voting or averages of predictions. Higher number of trees give you better performance but makes your code slower. You should choose as high value as your processor can handle because this makes your predictions stronger and more stable.
## tfidf__sublinear_tf
It seems unlikely that twenty occurrences of a term in a document truly carry twenty times the significance of a single occurrence. Accordingly, there has been considerable research into variants of term frequency that go beyond counting the number of occurrences of a term. A common modification is to use instead the logarithm of the term frequency.


In [17]:
parameters = {
    'clf__estimator__n_estimators': [100, 200, 300],
    # 'clf__estimator__criterion': ['gini', 'entropy', 'log_loss'],
    'features__text_pipeline__tfidf__sublinear_tf':[True, False]
    }

X_train, X_test, y_train, y_test = train_test_split(X, Y)

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
cv.fit(X_train.message, y_train)

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        FeatureUnion(transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('tfidf',
                                                                                         TfidfVectorizer(tokenizer=<function tokenize at 0x7faef8e5b8b0>))]))])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))]),
             param_grid={'clf__estimator__n_estimators': [100, 200, 300],
                         'features__text_pipeline__tfidf__sublinear_tf': [True,
                                                                          False]})

In [21]:
# dump best model 
import joblib
joblib.dump(cv.best_estimator_, 'cv_model.pkl')

['cv_model.pkl']

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [22]:
predicted = cv.predict(X_test.message)

In [24]:
print(cv.best_params_)

{'clf__estimator__n_estimators': 200, 'features__text_pipeline__tfidf__sublinear_tf': True}


In [23]:
for index, y_pred in enumerate(np.transpose(predicted)):
    y_true=y_test.iloc[:, index]
    print(f'{headers[index]}: {classification_report(y_true, y_pred)}')

related:               precision    recall  f1-score   support

           0       0.73      0.41      0.53      1577
           1       0.83      0.95      0.89      4924
           2       0.71      0.23      0.34        53

    accuracy                           0.82      6554
   macro avg       0.76      0.53      0.59      6554
weighted avg       0.80      0.82      0.80      6554

request:               precision    recall  f1-score   support

           0       0.91      0.98      0.94      5453
           1       0.83      0.50      0.63      1101

    accuracy                           0.90      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

offer:               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6529
           1       0.00      0.00      0.00        25

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      

/home/heiko/miniconda3/envs/test/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/heiko/miniconda3/envs/test/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/heiko/miniconda3/envs/test/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/

cold:               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6432
           1       0.92      0.09      0.16       122

    accuracy                           0.98      6554
   macro avg       0.95      0.55      0.58      6554
weighted avg       0.98      0.98      0.98      6554

other_weather:               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6207
           1       0.60      0.03      0.07       347

    accuracy                           0.95      6554
   macro avg       0.77      0.52      0.52      6554
weighted avg       0.93      0.95      0.93      6554

direct_report:               precision    recall  f1-score   support

           0       0.86      0.98      0.92      5296
           1       0.78      0.35      0.49      1258

    accuracy                           0.86      6554
   macro avg       0.82      0.67      0.70      6554
weighted avg       0.85      0.86     

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [21]:
model = joblib.load("models/classifier.pkl")

In [23]:
model.predict(['lost my family and need water'])

array([[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

: 